In [9]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb

In [3]:
df_train = pd.read_excel("../data/raw/entrenamiento.xlsx")
df_train = df_train.rename(columns={"C7.1": "C8", "des": "QUALITY"})


In [4]:
high_values_train = df_train["C8"] > 900
df_train.loc[high_values_train, "C8"] = df_train.loc[high_values_train, "C8"] / 1000

In [18]:
# Function to impute outliers using IQR and column median
def impute_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)

    # Impute outliers with the median of the column
    data.loc[outliers, column] = data[column].median()
    return data

# Impute outliers in the dataset using IQR method
df_imputed_outliers = df_train.copy()
for column in df_train.columns:
    if df_train[column].dtype != 'object':  # Apply only for numerical columns
        df_imputed_outliers = impute_outliers_iqr(df_imputed_outliers, column)

# Separate features and target
X = df_imputed_outliers.drop("QUALITY", axis=1)
y = df_imputed_outliers["QUALITY"]

# Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train_std, X_test_std, y_train, y_test = train_test_split(X_standardized, y, test_size=0.3, random_state=42)

# Initialize the XGBoost regressor with the optimized hyperparameters
xgb_optimized = {
    'objective': 'reg:squarederror',
    'colsample_bytree':0.8,
    'gamma':0,
    'learning_rate':0.1,
    'max_depth':8,
    'min_child_weight':1,
    'n_estimators':200,
    'subsample':0.8,
    'random_state':42
}

optimized_xgb = xgb.XGBRegressor(**xgb_optimized)
optimized_xgb.fit(X_train_std, y_train)
train_predictions_optimized = optimized_xgb.predict(X_train_std)
mae_optimized = mean_absolute_error(y_train, train_predictions_optimized)
print(f"Mean Absolute Error (MAE) con hiperparámetros optimizados: {mae_optimized:.4f}")

/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Mean Absolute Error (MAE) con hiperparámetros optimizados: 0.0330


In [19]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb

# Definir el modelo
xgb_model = xgb.XGBRegressor(
    subsample=0.8, 
    n_estimators=200, 
    min_child_weight=1, 
    max_depth=8, 
    learning_rate=0.1, 
    gamma=0, 
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

# Realizar validación cruzada con MAE como métrica de evaluación
scores = -cross_val_score(xgb_model, X_standardized, y, cv=5, scoring='neg_mean_absolute_error')

print(f"MAE scores from cross-validation: {scores}")
print(f"Mean MAE score: {scores.mean()}")
print(f"Standard Deviation of MAE scores: {scores.std()}")

/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/sit

MAE scores from cross-validation: [0.47587791 0.44130769 0.46185581 0.46181228 0.45416789]
Mean MAE score: 0.45900431441177
Standard Deviation of MAE scores: 0.011289775751033553


In [20]:
# Aumentar la regularización
xgb_reg_params = {
    'objective': 'reg:squarederror',
    'subsample': 0.8, 
    'n_estimators': 200, 
    'min_child_weight': 1, 
    'max_depth': 8, 
    'learning_rate': 0.05,  # Disminuir la tasa de aprendizaje
    'gamma': 0.5,          # Aumentar gamma
    'colsample_bytree': 0.8,
    'alpha': 0.5,          # Aumentar alpha
    'lambda': 1.5,         # Aumentar lambda
    'random_state': 42
}

# Entrenar el modelo con los nuevos parámetros
optimized_xgb_reg = xgb.XGBRegressor(**xgb_reg_params)
optimized_xgb_reg.fit(X_train_std, y_train)

# Evaluar el modelo en el conjunto de entrenamiento
train_predictions = optimized_xgb_reg.predict(X_train_std)
mae = mean_absolute_error(y_train, train_predictions)
print(f"Mean Absolute Error (MAE) con mayor regularización: {mae:.4f}")


/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Mean Absolute Error (MAE) con mayor regularización: 0.3018
